In [5]:
import polars as pl
import os
from pymongo import MongoClient

In [2]:
client = MongoClient(os.environ["mongodb_conn"])
db = client["Cluster0"]
collection = db["Cases"]

In [3]:
cursor = collection.find()        # returns a cursor
data   = list(cursor) 

In [7]:
df = pl.DataFrame({"Cases": data}).unnest("Cases")

In [15]:
df.sort(by = "TimeScraped", descending = False)["CaseID"].to_list()

['D 02 JV 25 0000092',
 'D 02 JV 25 0000145',
 'D 01 JV 25 0000454',
 'D 01 JV 25 0000211',
 'D 02 JV 25 0000003',
 'D 01 JV 25 0000356',
 'D 01 JV 25 0000360',
 'D 02 JV 25 0000032',
 'D 02 JV 25 0000006',
 'D 01 JV 25 0000105',
 'D 01 JV 25 0000250',
 'D 02 JV 25 0000300',
 'D 02 JV 25 0000050',
 'D 01 JV 25 0000106',
 'D 01 JV 25 0000307',
 'D 01 JV 25 0000120',
 'D 02 JV 25 0000069',
 'D 01 JV 25 0000182',
 'D 01 JV 25 0000453',
 'D 02 JV 25 0000247',
 'D 02 JV 25 0000114',
 'D 02 JV 25 0000278',
 'D 01 JV 25 0000287',
 'D 59 JV 25 0000173',
 'D 01 JV 25 0000247',
 'D 59 JV 25 0000033',
 'D 01 JV 25 0000216',
 'D 02 JV 25 0000035',
 'D 01 JV 25 0000209',
 'D 01 JV 25 0000188',
 'D 59 JV 25 0000168',
 'D 02 JV 25 0000049',
 'D 01 JV 25 0000347',
 'D 02 JV 25 0000078',
 'D 01 JV 25 0000030',
 'D 01 JV 25 0000348',
 'D 59 JV 25 0000164',
 'D 01 JV 25 0000400',
 'D 02 JV 25 0000182',
 'D 01 JV 25 0000252',
 'D 59 JV 25 0000113',
 'D 02 JV 25 0000009',
 'D 01 JV 25 0000263',
 'D 02 JV 2

In [10]:
for case in data:
    case_id = case["CaseID"]
    with open("./Cases/" + case_id + ".html", "w") as f:
        f.write(case["Docket"])

In [ ]:
from util import get_next_n_cases, parse_case_info
from datetime import date
import polars as pl
import os

def get_bounced_cases(MONGO_URI) -> pl.DataFrame:
    client = MongoClient(MONGO_URI)
    db = client["Cluster0"]
    collection = db["Cases"]

    cursor = collection.find()  
    data   = list(cursor) 

    not_scraped = [parse_case_info(d["CaseID"]) for d in data if "Case Summary" not in d["Docket"]]

    df = pl.DataFrame({"parsed": not_scraped})
    df = df.with_columns(
            pl.lit(date.today()).cast(pl.Datetime).alias("TimeScraped"),
            pl.lit(None).alias("Docket"),
            pl.lit(None).alias("DateOfBirth")
        ).unnest("parsed")
    
    print(df)

    return df

In [17]:
cursor = collection.find()        # returns a cursor

In [18]:
data   = list(cursor) 

In [20]:
not_scraped = [parse_case_info(d["CaseID"]) for d in data if "Case Summary" not in d["Docket"]]

In [21]:
df = pl.DataFrame({"parsed": not_scraped})
df = df.with_columns(
        pl.lit(date.today()).cast(pl.Datetime).alias("TimeScraped"),
        pl.lit(None).alias("Docket"),
        pl.lit(None).alias("DateOfBirth")
    ).unnest("parsed")

In [6]:
get_bounced_cases(os.environ["mongodb_conn"])

shape: (444, 6)
┌──────────┬───────────┬────────────┬─────────────────────┬────────┬─────────────┐
│ CaseYear ┆ County    ┆ CaseNumber ┆ TimeScraped         ┆ Docket ┆ DateOfBirth │
│ ---      ┆ ---       ┆ ---        ┆ ---                 ┆ ---    ┆ ---         │
│ i64      ┆ str       ┆ str        ┆ datetime[μs]        ┆ null   ┆ null        │
╞══════════╪═══════════╪════════════╪═════════════════════╪════════╪═════════════╡
│ 2025     ┆ Lancaster ┆ 0000092    ┆ 2025-05-28 00:00:00 ┆ null   ┆ null        │
│ 2025     ┆ Lancaster ┆ 0000145    ┆ 2025-05-28 00:00:00 ┆ null   ┆ null        │
│ 2025     ┆ Lancaster ┆ 0000003    ┆ 2025-05-28 00:00:00 ┆ null   ┆ null        │
│ 2025     ┆ Lancaster ┆ 0000032    ┆ 2025-05-28 00:00:00 ┆ null   ┆ null        │
│ …        ┆ …         ┆ …          ┆ …                   ┆ …      ┆ …           │
│ 2025     ┆ Douglas   ┆ 0000255    ┆ 2025-05-28 00:00:00 ┆ null   ┆ null        │
│ 2025     ┆ Lancaster ┆ 0000065    ┆ 2025-05-28 00:00:00 ┆ null   ┆ nu

shape: (444, 6)
┌──────────┬───────────┬────────────┬─────────────────────┬────────┬─────────────┐
│ CaseYear ┆ County    ┆ CaseNumber ┆ TimeScraped         ┆ Docket ┆ DateOfBirth │
│ ---      ┆ ---       ┆ ---        ┆ ---                 ┆ ---    ┆ ---         │
│ i64      ┆ str       ┆ str        ┆ datetime[μs]        ┆ null   ┆ null        │
╞══════════╪═══════════╪════════════╪═════════════════════╪════════╪═════════════╡
│ 2025     ┆ Lancaster ┆ 0000092    ┆ 2025-05-28 00:00:00 ┆ null   ┆ null        │
│ 2025     ┆ Lancaster ┆ 0000145    ┆ 2025-05-28 00:00:00 ┆ null   ┆ null        │
│ 2025     ┆ Lancaster ┆ 0000003    ┆ 2025-05-28 00:00:00 ┆ null   ┆ null        │
│ 2025     ┆ Lancaster ┆ 0000032    ┆ 2025-05-28 00:00:00 ┆ null   ┆ null        │
│ …        ┆ …         ┆ …          ┆ …                   ┆ …      ┆ …           │
│ 2025     ┆ Douglas   ┆ 0000255    ┆ 2025-05-28 00:00:00 ┆ null   ┆ null        │
│ 2025     ┆ Lancaster ┆ 0000065    ┆ 2025-05-28 00:00:00 ┆ null   ┆ null        │
│ 2025     ┆ Douglas   ┆ 0000369    ┆ 2025-05-28 00:00:00 ┆ null   ┆ null        │
│ 2025     ┆ Lancaster ┆ 0000002    ┆ 2025-05-28 00:00:00 ┆ null   ┆ null        │
└──────────┴───────────┴────────────┴─────────────────────┴────────┴─────────────┘

In [18]:
get_next_n_cases(os.environ["mongodb_conn"])

shape: (60, 7)
┌────────────────────┬──────────┬─────────┬────────────┬──────────────┬────────┬─────────────┐
│ CaseID             ┆ CaseYear ┆ County  ┆ CaseNumber ┆ TimeScraped  ┆ Docket ┆ DateOfBirth │
│ ---                ┆ ---      ┆ ---     ┆ ---        ┆ ---          ┆ ---    ┆ ---         │
│ str                ┆ i64      ┆ str     ┆ str        ┆ datetime[μs] ┆ null   ┆ null        │
╞════════════════════╪══════════╪═════════╪════════════╪══════════════╪════════╪═════════════╡
│ D 01 JV 25 0000646 ┆ 2025     ┆ Douglas ┆ 0000646    ┆ 2025-05-28   ┆ null   ┆ null        │
│                    ┆          ┆         ┆            ┆ 00:00:00     ┆        ┆             │
│ D 01 JV 25 0000647 ┆ 2025     ┆ Douglas ┆ 0000647    ┆ 2025-05-28   ┆ null   ┆ null        │
│                    ┆          ┆         ┆            ┆ 00:00:00     ┆        ┆             │
│ D 01 JV 25 0000648 ┆ 2025     ┆ Douglas ┆ 0000648    ┆ 2025-05-28   ┆ null   ┆ null        │
│                    ┆          ┆  

shape: (60, 7)
┌────────────────────┬──────────┬─────────┬────────────┬──────────────┬────────┬─────────────┐
│ CaseID             ┆ CaseYear ┆ County  ┆ CaseNumber ┆ TimeScraped  ┆ Docket ┆ DateOfBirth │
│ ---                ┆ ---      ┆ ---     ┆ ---        ┆ ---          ┆ ---    ┆ ---         │
│ str                ┆ i64      ┆ str     ┆ str        ┆ datetime[μs] ┆ null   ┆ null        │
╞════════════════════╪══════════╪═════════╪════════════╪══════════════╪════════╪═════════════╡
│ D 01 JV 25 0000646 ┆ 2025     ┆ Douglas ┆ 0000646    ┆ 2025-05-28   ┆ null   ┆ null        │
│                    ┆          ┆         ┆            ┆ 00:00:00     ┆        ┆             │
│ D 01 JV 25 0000647 ┆ 2025     ┆ Douglas ┆ 0000647    ┆ 2025-05-28   ┆ null   ┆ null        │
│                    ┆          ┆         ┆            ┆ 00:00:00     ┆        ┆             │
│ D 01 JV 25 0000648 ┆ 2025     ┆ Douglas ┆ 0000648    ┆ 2025-05-28   ┆ null   ┆ null        │
│                    ┆          ┆         ┆            ┆ 00:00:00     ┆        ┆             │
│ D 01 JV 25 0000649 ┆ 2025     ┆ Douglas ┆ 0000649    ┆ 2025-05-28   ┆ null   ┆ null        │
│                    ┆          ┆         ┆            ┆ 00:00:00     ┆        ┆             │
│ …                  ┆ …        ┆ …       ┆ …          ┆ …            ┆ …      ┆ …           │
│ D 59 JV 25 0000251 ┆ 2025     ┆ Sarpy   ┆ 0000251    ┆ 2025-05-28   ┆ null   ┆ null        │
│                    ┆          ┆         ┆            ┆ 00:00:00     ┆        ┆             │
│ D 59 JV 25 0000252 ┆ 2025     ┆ Sarpy   ┆ 0000252    ┆ 2025-05-28   ┆ null   ┆ null        │
│                    ┆          ┆         ┆            ┆ 00:00:00     ┆        ┆             │
│ D 59 JV 25 0000253 ┆ 2025     ┆ Sarpy   ┆ 0000253    ┆ 2025-05-28   ┆ null   ┆ null        │
│                    ┆          ┆         ┆            ┆ 00:00:00     ┆        ┆             │
│ D 59 JV 25 0000254 ┆ 2025     ┆ Sarpy   ┆ 0000254    ┆ 2025-05-28   ┆ null   ┆ null        │
│                    ┆          ┆         ┆            ┆ 00:00:00     ┆        ┆             │
└────────────────────┴──────────┴─────────┴────────────┴──────────────┴────────┴─────────────┘